In [ ]:
import os
os.environ["OPENROUTER_API_KEY"] = "<PASTE_HERE_YOUR_OPEN_ROUTER_API_KEY>"

In [ ]:
# AI-Assisted Error Debugger with Model Fallback
import os
import requests
import traceback
import re
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, Markdown
from IPython import get_ipython

# ==============================
# 1. OpenRouter config
# ==============================
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_ENDPOINT = "https://openrouter.ai/api/v1/chat/completions"

if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY is not set in environment variables!")

# Priority list of free models (best → fallback)
MODELS = [
    "x-ai/grok-4-fast:free",
    "deepseek/deepseek-r1:free",
    "deepseek/deepseek-chat-v3-0324:free",
    "meta-llama/llama-4-maverick:free",
    "meta-llama/llama-4-scout:free",
    "openai/gpt-oss-120b:free"
]

# ==============================
# 2. Call AI via OpenRouter with fallback
# ==============================
def ask_ai_for_fix(code, error_trace):
    prompt = f"""
You are a Python expert. The following code produced an error:

```python
{code}
```

Error traceback:

{error_trace}

Instructions:
1. Explain why the error happened.
2. Suggest a fix if possible.
3. ONLY provide corrected Python code in a single ```python``` block.
4. If the code is already correct, just say "No correction needed".
"""

    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
    }

    # Try each model in priority order
    for model in MODELS:
        payload = {
            "model": model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0,
        }
        try:
            response = requests.post(OPENROUTER_ENDPOINT, json=payload, headers=headers, timeout=30)
            response.raise_for_status()
            result = response.json()
            return result["choices"][0]["message"]["content"] + f"\n\n*(Model used: {model})*"
        except Exception as e:
            # Try next model
            last_error = str(e)
            continue

    return f"AI request failed for all models. Last error: {last_error}"

# ==============================
# 3. Custom exception handler
# ==============================
def ai_exception_handler(shell, etype, evalue, tb, tb_offset=None):
    tb_str = "".join(traceback.format_exception(etype, evalue, tb))

    # Grab last executed cell input safely
    try:
        code = shell.user_ns.get("_ih", [""])[-1]
    except Exception:
        code = ""

    ai_response = ask_ai_for_fix(code, tb_str)

    display(Markdown("### ⚡ AI Debugging Suggestion"))
    display(Markdown(ai_response))

    # Extract corrected code block
    match = re.search(r"```python\n(.*?)```", ai_response, re.DOTALL)
    if match:
        corrected_code = match.group(1).strip()
        if corrected_code and corrected_code != code.strip():
            shell.set_next_input(corrected_code, replace=False)

# ==============================
# 4. Activate AI handler
# ==============================
ip = get_ipython()
ip.set_custom_exc((Exception,), ai_exception_handler)

display(Markdown("**AI-assisted error handling with model fallback is now active!**\nTry running code with an error."))


In [ ]:
fo i in [6,7]:
    print(i)

In [ ]:
result = "Age: " + 25
print(result)


In [ ]:
# Cell 1: Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Cell 2: Load dataset (simulated real-world)
data = pd.DataFrame({
    'CustomerID': [101, 102, 103, 104, 105],
    'Age': [25, 30, 'Thirty-Five', 40, 29],  # One value as string
    'Tenure': [1, 3, 5, 2, 4],
    'Balance': [5000, 7000, 12000, 'NaN', 4000],  # One value as string 'NaN'
    'Churn': [0, 1, 0, 1, 0]
})

# Cell 3: Preprocessing
# Convert Age and Balance to numeric
data['Age'] = data['Age'].astype(int)          # Will fail due to 'Thirty-Five'
data['Balance'] = pd.to_numeric(data['Balance'])  # Will fail due to 'NaN' string

# Scale numeric features
scaler = StandardScaler()
data[['Age', 'Tenure', 'Balance']] = scaler.fit_transform(data[['Age', 'Tenure', 'Balance']])

# Cell 4: Train/Test split
X = data[['Age', 'Tenure', 'Balance']]
y = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Cell 5: Model training
model = LogisticRegression()
model.fit(X_train, y_train)  # Might fail if preprocessing didn't run

# Cell 6: Prediction & evaluation
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {acc:.2f}")